In [1]:
backend = "pt_backend"

In [2]:
import time
import numpy as np

np.random.seed(0)

match backend:
    case "original_backend":
        from lib.original_backend.linear_algebra import Matrix
        from lib.original_backend.nn import  NN, ReLU, Linear, LayerNorm
        from lib.original_backend.processing import ColumnNormalizer
        from lib.optimizers import SgdOptimizer, SgdWithMomentumOptimizer, AdaGradOptimizer, RmsPropOptimizer, AdamOptimizer

    case "np_backend":
        from lib.np_backend.linear_algebra import Matrix
        from lib.np_backend.nn import NN, ReLU, Linear, LayerNorm
        from lib.np_backend.processing import ColumnNormalizer
        from lib.optimizers import SgdOptimizer, SgdWithMomentumOptimizer, AdaGradOptimizer, RmsPropOptimizer, AdamOptimizer

    case "pt_backend":
        from lib.pt_backend.linear_algebra import Matrix
        from lib.pt_backend.nn import NN, ReLU, Linear, LayerNorm
        from lib.pt_backend.processing import ColumnNormalizer
        from lib.pt_backend.optimizers import SgdOptimizer, SgdWithMomentumOptimizer, AdaGradOptimizer, RmsPropOptimizer, AdamOptimizer

from lib.metrics.losses import mean_squared_error
from lib.gd_data_loaders import BatchDataLoader, StochasticDataLoader, MiniBatchDataLoader

In [3]:
# Boston House Price Dataset
# The Boston House Price Dataset involves the prediction of a house price in thousands of dollars given details of the house and its neighborhood.
# It is a regression problem. There are 506 observations with 13 input variables and 1 output variable. The variable names are as follows:

data = []
with open("data/boston_house_prices.data", "rt") as f:
    for line in f.readlines():
        data.append([float(v) for v in line.split()])
data = np.array(data)

In [4]:
np.random.shuffle(data)
split = int(len(data) * 0.8)
train_set = data[:split]
test_set = data[split:]

X_train, y_train = train_set[:, :-1], train_set[:, -1:]
X_test, y_test = test_set[:, :-1], test_set[:, -1:]
X_train = Matrix(X_train)
y_train = Matrix(y_train)
X_test = Matrix(X_test)
y_test = Matrix(y_test)
X_train.dims(), y_train.dims(), X_test.dims(), y_test.dims()

(torch.Size([404, 13]),
 torch.Size([404, 1]),
 torch.Size([102, 13]),
 torch.Size([102, 1]))

In [5]:
normalizer = ColumnNormalizer()
normalizer.fit(X_train)
X_train = normalizer.transform(X_train)
X_test = normalizer.transform(X_test)
X_train.dims(), X_test.dims()

(torch.Size([404, 13]), torch.Size([102, 13]))

In [6]:
def init_nn():
    return NN([
        Linear(13, 4),
        LayerNorm(4),
        ReLU(),
        Linear(4, 1),
    ])

In [7]:
time_point = time.time()

data_loaders = [
    # BatchDataLoader(X_train, y_train),
    # StochasticDataLoader(X_train, y_train),
    MiniBatchDataLoader(X_train, y_train, 32)
]
optimizer_creators = [
    # lambda nn: SgdOptimizer(nn, 0.001),
    # lambda nn: SgdWithMomentumOptimizer(nn, 0.001, 0.9),
    lambda nn: AdaGradOptimizer(nn, 0.05),
    lambda nn: RmsPropOptimizer(nn, 0.01, 0.95),
    lambda nn: AdamOptimizer(nn, 0.01, 0.95, 0.95),
]

for data_loader in data_loaders:
    for optimizer_creator in optimizer_creators:
        nn = init_nn()
        optimizer = optimizer_creator(nn)
        print(f"gradient descent: {data_loader.__class__} | optimizer: {optimizer.__class__}")
        for i in range(2001):
            X_b, y_b = data_loader.get_batch()
            out = nn(X_b)
            loss = mean_squared_error(y_b, out)
            if i % 100 == 0:
                elapsed_time = int(time.time() - time_point)
                time_point = time.time()
                print(f"{i} | {loss.data:.2f} | {elapsed_time}s") 

            optimizer.step(loss)

        train_out = nn(X_train) 
        train_loss = mean_squared_error(y_train, train_out)
        test_out = nn(X_test) 
        test_loss = mean_squared_error(y_test, test_out)
        print(f"train loss: {train_loss.data:.2f}   test loss: {test_loss.data:.2f}") 

gradient descent: <class 'lib.gd_data_loaders.MiniBatchDataLoader'> | optimizer: <class 'lib.pt_backend.optimizers.AdaGradOptimizer'>
0 | 575.62 | 0s
100 | 146.36 | 0s
200 | 143.13 | 0s
300 | 65.19 | 0s
400 | 59.04 | 0s
500 | 76.57 | 0s
600 | 20.46 | 0s
700 | 54.61 | 0s
800 | 24.13 | 0s
900 | 38.33 | 0s
1000 | 20.70 | 0s
1100 | 17.65 | 0s
1200 | 18.01 | 0s
1300 | 45.34 | 0s
1400 | 15.03 | 0s
1500 | 37.47 | 0s
1600 | 11.69 | 0s
1700 | 23.82 | 0s
1800 | 10.93 | 0s
1900 | 7.27 | 0s
2000 | 7.63 | 0s
train loss: 16.16   test loss: 20.90
gradient descent: <class 'lib.gd_data_loaders.MiniBatchDataLoader'> | optimizer: <class 'lib.pt_backend.optimizers.RmsPropOptimizer'>
0 | 655.30 | 0s
100 | 141.01 | 0s
200 | 70.01 | 0s
300 | 30.71 | 0s
400 | 15.11 | 0s
500 | 9.69 | 0s
600 | 8.05 | 0s
700 | 16.61 | 0s
800 | 8.11 | 0s
900 | 12.95 | 0s
1000 | 17.44 | 0s
1100 | 3.89 | 0s
1200 | 13.50 | 0s
1300 | 14.28 | 0s
1400 | 18.07 | 0s
1500 | 22.07 | 0s
1600 | 9.31 | 0s
1700 | 5.96 | 0s
1800 | 14.77 | 0s
19

In [8]:
[(round(float(v1),1), float(v2)) for v1, v2 in zip([v[0].data for v in out], [v[0].data for v in y_test])][:10]

[(32.2, 20.700000762939453),
 (11.2, 39.79999923706055),
 (21.4, 17.799999237060547),
 (27.2, 19.600000381469727),
 (23.5, 14.899999618530273),
 (22.0, 22.0),
 (20.6, 48.79999923706055),
 (15.4, 25.0),
 (21.4, 48.5),
 (31.8, 23.899999618530273)]